In [1]:
# English + Bengali

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 🔸 Synthetic classification dataset তৈরি করছি
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# 🔸 Feature scaling (Standardization)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 🔸 Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔸 Numpy → Tensor conversion
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test  = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# 🔸 ANN model তৈরি করছি (PyTorch nn.Module subclass)
class ANN(nn.Module):
    def __init__(self, input_dim):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)     # Hidden layer 1
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 16)            # Hidden layer 2
        self.fc3 = nn.Linear(16, 1)             # Output layer
        self.sigmoid = nn.Sigmoid()            # For binary classification

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

# 🔸 মডেল তৈরি
model = ANN(input_dim=20)

# 🔸 Loss function & Optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 🔸 Model training
epochs = 100
for epoch in range(epochs):
    model.train()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# 🔸 Model evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_label = (y_pred > 0.5).float()
    accuracy = (y_pred_label == y_test).sum() / y_test.shape[0]
    print(f"\nAccuracy: {accuracy.item()*100:.2f}%")


Epoch 10/100, Loss: 0.6836
Epoch 20/100, Loss: 0.6647
Epoch 30/100, Loss: 0.6379
Epoch 40/100, Loss: 0.6007
Epoch 50/100, Loss: 0.5525
Epoch 60/100, Loss: 0.4963
Epoch 70/100, Loss: 0.4382
Epoch 80/100, Loss: 0.3849
Epoch 90/100, Loss: 0.3406
Epoch 100/100, Loss: 0.3076

Accuracy: 83.50%
